### pip install & import

In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import pandas as pd
#import pandas_profiling
import datetime

# 예지언니 크롤링 데이터 머지

### 데이터 불러오기

In [ ]:
movie_resize_ALL = pd.read_csv('../2_resizing/resizing_all.csv',encoding='utf-8')
movie_resize_회귀 = pd.read_csv('../2_resizing/resizing_reg.csv',encoding='utf-8')
movie_resize_분류 = pd.read_csv('../2_resizing/resizing_cls.csv',encoding='utf-8')

### 크롤링된 파일 불러오기

In [ ]:
# 네이버 평점과 배우 가져옴

df2 = pd.read_csv("../../data/movie_data/concat.csv", encoding='UTF-8',index_col=0)
df2 = df2.drop(['네이버URL','이미지URL','제작연도','감독','연도'],axis=1)

### 빈칸 있는 데이터 삭제

In [ ]:
# null 값이 있는 열 삭제 
df2 = df2[df2['출연'].isnull() == False] 
df2.info()

### movie_resize 데이터 + 크롤링된 데이터 Merge

In [ ]:
movie_resize4 = pd.merge(movie_resize_ALL,df2,on='영화명')
movie_resize5 = pd.merge(movie_resize_회귀,df2,on='영화명')
movie_resize6 = pd.merge(movie_resize_분류,df2,on='영화명')

### 중복 데이터 삭제

In [ ]:
movie_resize4.drop_duplicates('영화명',keep='first',inplace=True)
movie_resize4.info()

movie_resize5.drop_duplicates('영화명',keep='first',inplace=True)
movie_resize5.info()

movie_resize6.drop_duplicates('영화명',keep='first',inplace=True)
movie_resize6.info()

In [ ]:
# Unnamed: 0 삭제하는 법
# drop 할 때에는 list에 넣어서 한다.

movie_resize4.drop(['Unnamed: 0'],axis=1,inplace=True)
movie_resize5.drop(['Unnamed: 0'],axis=1,inplace=True)
movie_resize6.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
# 중복 데이터 없어졌는 지 확인
movie_resize4[movie_resize4.duplicated()==True]
movie_resize5[movie_resize5.duplicated()==True]
movie_resize6[movie_resize6.duplicated()==True]

In [ ]:
# 변경 될때만 따로 풀어서 사용하시길.(파일생성코드)
movie_resize4.to_csv('./movie_resize_merged_all.csv',encoding='utf-8')
movie_resize5.to_csv('./movie_resize_merged_reg.csv',encoding='utf-8')
movie_resize6.to_csv('./movie_resize_merged_cls.csv',encoding='utf-8')

# movie_resize_merged.csv에서 top 300 생성

### Merge된 데이터 불러오기

In [ ]:
movie_resize_all = pd.read_csv('./movie_resize_merged_all.csv',encoding='utf-8')
movie_resize_회귀 = pd.read_csv('./movie_resize_merged_reg.csv',encoding='utf-8')
movie_resize_분류 = pd.read_csv('./movie_resize_merged_cls.csv',encoding='utf-8')

### Top 300 저장 및 불러오기

In [ ]:
# 변경 될때만 따로 풀어서 사용하시길.(파일생성코드)
col = ['영화명', '감독', '배급사', '개봉일', '출연']
movie_resize_all.reset_index(drop=False,inplace=True)
movie_resize_all[col][:300].to_csv(
    './movie_resize_ranking.csv', encoding="utf-8")


In [ ]:
# top 300 불러오기

movie_resize_ranking = pd.read_csv('./movie_resize_ranking.csv',encoding='UTF-8')
movie_resize_ranking.drop(['Unnamed: 0'], axis=1, inplace=True)
movie_resize_ranking

In [ ]:
# 점수화를 위해 movie_resize_ranking['개봉일']을 datetime 타입으로 변경

movie_resize_ranking['개봉일'] = movie_resize_ranking['개봉일'].astype('datetime64')

### 점수화행 만들기

In [ ]:
# 점수화행 만들기
movie_resize_회귀['개봉일'] = movie_resize_회귀['개봉일'].astype('datetime64')

movie_final_회귀 = movie_resize_회귀.copy()
movie_final_회귀['감독_흥행'] = [0 for _ in range(len(movie_resize_회귀))]
movie_final_회귀['배급사_흥행'] = [0 for _ in range(len(movie_resize_회귀))]
movie_final_회귀['주연배우_흥행'] = [0 for _ in range(len(movie_resize_회귀))]

In [ ]:
# 점수화행 만들기
movie_resize_분류['개봉일'] = movie_resize_분류['개봉일'].astype('datetime64')

movie_final_분류 = movie_resize_분류.copy()
movie_final_분류['감독_흥행'] = [0 for _ in range(len(movie_resize_분류))]
movie_final_분류['배급사_흥행'] = [0 for _ in range(len(movie_resize_분류))]
movie_final_분류['주연배우_흥행'] = [0 for _ in range(len(movie_resize_분류))]

In [ ]:
# 순번을 쓰기 위해 인덱스 풀기

movie_resize_ranking.reset_index(drop=False,inplace=True)

### 감독, 배급사 점수화 코드

### 회귀 모델 점수화

In [ ]:
# 감독 점수화 코드

for idx, row in movie_final_회귀.iterrows():
    # 개봉일 기준
    open = datetime.datetime.strftime(row['개봉일'], '%Y-%m-%d')
    director = row['감독'].strip()
    # Top300 중 개봉일 이전 영화의 감독 확인
    directors = movie_resize_ranking[(movie_resize_ranking['개봉일'] < open) & (movie_resize_ranking['감독'] == director)]
    # 해당 감독이 있다면 흥행 여부 표시
    if len(directors) > 0:
        score = directors.index[0]
        if score >= 271:
            score = 1
        elif score >= 241:
            score = 2
        elif score >= 211:
            score = 3
        elif score >= 181:
            score = 4
        elif score >= 151:
            score = 5
        elif score >= 121:
            score = 6
        elif score >= 91:
            score = 7
        elif score >= 61:
            score = 8
        elif score >= 31:
            score = 9
        else:
            score = 10
        movie_final_회귀.loc[idx, '감독_흥행'] = score

In [ ]:
# 배급사 정수화 코드 
for idx, row in movie_final_회귀.iterrows():
    # 개봉일 기준
    open = datetime.datetime.strftime(row['개봉일'], '%Y-%m-%d')
    director = row['배급사'].strip()
    # Top300 중 개봉일 이전 영화의 감독 확인
    directors = movie_resize_ranking[(movie_resize_ranking['개봉일'] < open) & (movie_resize_ranking['배급사'] == director)]
    # 해당 감독이 있다면 흥행 여부 표시
    if len(directors) > 0:
        score = directors.index[0]
        if score >= 271:
            score = 1
        elif score >= 241:
            score = 2
        elif score >= 211:
            score = 3
        elif score >= 181:
            score = 4
        elif score >= 151:
            score = 5
        elif score >= 121:
            score = 6
        elif score >= 91:
            score = 7
        elif score >= 61:
            score = 8
        elif score >= 31:
            score = 9
        else:
            score = 10
        movie_final_회귀.loc[idx, '배급사_흥행'] = score

In [ ]:
# 배우의 경우 '["배우1", "배우2"]'와 같은 형태이므로 각 값을 뽑아낸다.
# literal_evel : 문자열 그대로 파이썬 명령을 실행시켜주는 함수
# 예시---------------------------------------
# 문자열 : "['문자열1', '문자열2', 22, '33']"
# 리스트 : ['문자열1', '문자열2', 22, '33']
from ast import literal_eval

for idx, row in movie_final_회귀.iterrows():
    # 개봉일 기준
    open = datetime.datetime.strftime(row['개봉일'], '%Y-%m-%d')
    # 해당 영화의 배우 확인
    this_actors = literal_eval(row['출연'])
    scores = [0] * len(this_actors)
    # 개봉일 이전 영화의 모든 배우 확인
    for i in range(len(this_actors)):
        actors = movie_resize_ranking.query(f'개봉일 < "{open}"')
        actors = actors.query(
            f'출연.str.contains("{this_actors[i]}")', engine='python')
        if len(actors) > 0:
            score = actors.index[0]
            if score >= 271:
                score = 1
            elif score >= 241:
                score = 2
            elif score >= 211:
                score = 3
            elif score >= 181:
                score = 4
            elif score >= 151:
                score = 5
            elif score >= 121:
                score = 6
            elif score >= 91:
                score = 7
            elif score >= 61:
                score = 8
            elif score >= 31:
                score = 9
            else:
                score = 10
            scores[i] = score

    # 평균/합/최대값 중에 잘 나오는 걸로        
    # 평균값
    # movie_final.loc[idx, '주연배우_흥행'] = round(sum(scores) / len(scores), 1)
    # 합
    movie_final_회귀.loc[idx, '주연배우_흥행'] = sum(scores)
    # 최대값
    # movie_final.loc[idx, '주연배우_흥행'] = max(scores)


In [ ]:
# 값 확인
print((movie_final_회귀['감독_흥행'] == 0).value_counts())
print()
print((movie_final_회귀['배급사_흥행'] == 0).value_counts())
print()
print((movie_final_회귀['주연배우_흥행'] == 0).value_counts())

### 분류 모델 점수화

In [ ]:
# 감독 점수화 코드

for idx, row in movie_final_분류.iterrows():
    # 개봉일 기준
    open = datetime.datetime.strftime(row['개봉일'], '%Y-%m-%d')
    director = row['감독'].strip()
    # Top300 중 개봉일 이전 영화의 감독 확인
    directors = movie_resize_ranking[(movie_resize_ranking['개봉일'] < open) & (movie_resize_ranking['감독'] == director)]
    # 해당 감독이 있다면 흥행 여부 표시
    if len(directors) > 0:
        score = directors.index[0]
        if score >= 271:
            score = 1
        elif score >= 241:
            score = 2
        elif score >= 211:
            score = 3
        elif score >= 181:
            score = 4
        elif score >= 151:
            score = 5
        elif score >= 121:
            score = 6
        elif score >= 91:
            score = 7
        elif score >= 61:
            score = 8
        elif score >= 31:
            score = 9
        else:
            score = 10
        movie_final_분류.loc[idx, '감독_흥행'] = score

In [ ]:
# 배급사 정수화 코드 
for idx, row in movie_final_분류.iterrows():
    # 개봉일 기준
    open = datetime.datetime.strftime(row['개봉일'], '%Y-%m-%d')
    director = row['배급사'].strip()
    # Top300 중 개봉일 이전 영화의 감독 확인
    directors = movie_resize_ranking[(movie_resize_ranking['개봉일'] < open) & (movie_resize_ranking['배급사'] == director)]
    # 해당 감독이 있다면 흥행 여부 표시
    if len(directors) > 0:
        score = directors.index[0]
        if score >= 271:
            score = 1
        elif score >= 241:
            score = 2
        elif score >= 211:
            score = 3
        elif score >= 181:
            score = 4
        elif score >= 151:
            score = 5
        elif score >= 121:
            score = 6
        elif score >= 91:
            score = 7
        elif score >= 61:
            score = 8
        elif score >= 31:
            score = 9
        else:
            score = 10
        movie_final_분류.loc[idx, '배급사_흥행'] = score

In [ ]:
# 배우의 경우 '["배우1", "배우2"]'와 같은 형태이므로 각 값을 뽑아낸다.
# literal_evel : 문자열 그대로 파이썬 명령을 실행시켜주는 함수
# 예시---------------------------------------
# 문자열 : "['문자열1', '문자열2', 22, '33']"
# 리스트 : ['문자열1', '문자열2', 22, '33']
from ast import literal_eval

for idx, row in movie_final_분류.iterrows():
    # 개봉일 기준
    open = datetime.datetime.strftime(row['개봉일'], '%Y-%m-%d')
    # 해당 영화의 배우 확인
    this_actors = literal_eval(row['출연'])
    scores = [0] * len(this_actors)
    # 개봉일 이전 영화의 모든 배우 확인
    for i in range(len(this_actors)):
        actors = movie_resize_ranking.query(f'개봉일 < "{open}"')
        actors = actors.query(
            f'출연.str.contains("{this_actors[i]}")', engine='python')
        if len(actors) > 0:
            score = actors.index[0]
            if score >= 271:
                score = 1
            elif score >= 241:
                score = 2
            elif score >= 211:
                score = 3
            elif score >= 181:
                score = 4
            elif score >= 151:
                score = 5
            elif score >= 121:
                score = 6
            elif score >= 91:
                score = 7
            elif score >= 61:
                score = 8
            elif score >= 31:
                score = 9
            else:
                score = 10
            scores[i] = score

    # 평균/합/최대값 중에 잘 나오는 걸로        
    # 평균값
    # movie_final.loc[idx, '주연배우_흥행'] = round(sum(scores) / len(scores), 1)
    # 합
    movie_final_분류.loc[idx, '주연배우_흥행'] = sum(scores)
    # 최대값
    # movie_final.loc[idx, '주연배우_흥행'] = max(scores)


In [ ]:
# 값 확인
print((movie_final_분류['감독_흥행'] == 0).value_counts())
print()
print((movie_final_분류['배급사_흥행'] == 0).value_counts())
print()
print((movie_final_분류['주연배우_흥행'] == 0).value_counts())

### movie final 저장

In [ ]:
movie_final_회귀.drop(['Unnamed: 0'],axis=1,inplace=True)
movie_final_분류.drop(['Unnamed: 0'],axis=1,inplace=True)

In [ ]:
# 변경 될때만 따로 풀어서 사용하시길.(파일생성코드)
# movie_final_회귀.to_csv('./movie_final_reg.csv',encoding='utf-8')
# movie_final_분류.to_csv('./movie_final_cls.csv',encoding='utf-8')